In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import math

In [2]:
# Set the output folder path where you want to save the text files
# output_folder_path = '/home/paulc600/local/HYPE Inputs/'

In [3]:
# read the csv file with SLC, soil and landuse info
slc = pd.read_csv('/home/paulc600/SMM/HYPE_geospatial/SLC_frac.csv')

In [4]:
slc

,soil code,7,4,7.1,4.1,5,7.2,7.3,5.1,7.4,...,8.9,4.6,9.7,9.8,5.2,1.2,4.7,6.2,6.3,6.4
0,Landuse code,10.000000,10.000000,12.000000,12.000000,10.000000,4.0,8.000000,12.000000,13.0,...,16.0,8.0,4.0,5.0,13.0,16.0,9.0,17.0,16.0,11.0
1,COMID\SLC,1007.000000,1004.000000,1207.000000,1204.000000,1005.000000,407.0,807.000000,1205.000000,1307.0,...,1608.0,804.0,409.0,509.0,1305.0,1601.0,904.0,1706.0,1606.0,1106.0
2,71040065,0.209138,0.000000,0.000000,0.000000,0.000000,0.0,0.333977,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,71039761,0.122792,0.054986,0.458405,0.362678,0.000000,0.0,0.001140,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,71046901,0.404110,0.000000,0.321918,0.000000,0.000000,0.0,0.160959,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,74001169,0.001280,0.875240,0.000000,0.015995,0.064619,0.0,0.000000,0.010237,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1388,74001168,0.014536,0.711890,0.000000,0.047335,0.100634,0.0,0.000000,0.021990,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,74001167,0.125176,0.710267,0.000000,0.056259,0.075949,0.0,0.000000,0.023910,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1390,74001166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# remove soil code from index
slc = slc.reset_index()
slc.loc[-1] = slc.columns
slc.sort_index(inplace=True)

In [6]:
slc.drop('index', axis=1, inplace=True)

In [7]:
slc

,soil code,7,4,7.1,4.1,5,7.2,7.3,5.1,7.4,...,8.9,4.6,9.7,9.8,5.2,1.2,4.7,6.2,6.3,6.4
-1,soil code,7,4,7.1,4.1,5,7.2,7.3,5.1,7.4,...,8.9,4.6,9.7,9.8,5.2,1.2,4.7,6.2,6.3,6.4
0,Landuse code,10.0,10.0,12.0,12.0,10.0,4.0,8.0,12.0,13.0,...,16.0,8.0,4.0,5.0,13.0,16.0,9.0,17.0,16.0,11.0
1,COMID\SLC,1007.0,1004.0,1207.0,1204.0,1005.0,407.0,807.0,1205.0,1307.0,...,1608.0,804.0,409.0,509.0,1305.0,1601.0,904.0,1706.0,1606.0,1106.0
2,71040065,0.209138,0.0,0.0,0.0,0.0,0.0,0.333977,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,71039761,0.122792,0.054986,0.458405,0.362678,0.0,0.0,0.00114,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,74001169,0.00128,0.87524,0.0,0.015995,0.064619,0.0,0.0,0.010237,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1388,74001168,0.014536,0.71189,0.0,0.047335,0.100634,0.0,0.0,0.02199,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,74001167,0.125176,0.710267,0.0,0.056259,0.075949,0.0,0.0,0.02391,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1390,74001166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove the three rows from the original dataset
removed_slc = slc.iloc[:3]

In [9]:
removed_slc

,soil code,7,4,7.1,4.1,5,7.2,7.3,5.1,7.4,...,8.9,4.6,9.7,9.8,5.2,1.2,4.7,6.2,6.3,6.4
-1,soil code,7,4,7.1,4.1,5,7.2,7.3,5.1,7.4,...,8.9,4.6,9.7,9.8,5.2,1.2,4.7,6.2,6.3,6.4
0,Landuse code,10.0,10.0,12.0,12.0,10.0,4.0,8.0,12.0,13.0,...,16.0,8.0,4.0,5.0,13.0,16.0,9.0,17.0,16.0,11.0
1,COMID\SLC,1007.0,1004.0,1207.0,1204.0,1005.0,407.0,807.0,1205.0,1307.0,...,1608.0,804.0,409.0,509.0,1305.0,1601.0,904.0,1706.0,1606.0,1106.0


In [10]:
# Transpose the DataFrame
slc_transposed = removed_slc.transpose().reset_index(drop=True)

In [11]:
# Set the first row as the index header
slc_transposed.set_axis(slc_transposed.iloc[0], axis=1, inplace=True)
slc_transposed = slc_transposed[1:]

In [12]:
# re order columns
new_order=['COMID\SLC', 'Landuse code', 'soil code']

In [13]:
# Rearrange the columns in the DataFrame
slc_transposed = slc_transposed[new_order]

In [14]:
# Convert all columns to int by truncating decimals
slc_transposed = slc_transposed.applymap(lambda x: int(float(x)) if isinstance(x, str) and '.' in x else int(x))

In [15]:
#Rename the columns
slc_transposed.rename(columns={'COMID\SLC':'SLC' , 'soil code':'SOIL TYPE', 'Landuse code':'LULC'}, inplace=True)   

In [16]:
slc_transposed

,SLC,LULC,SOIL TYPE
1,1007,10,7
2,1004,10,4
3,1207,12,7
4,1204,12,4
5,1005,10,5
6,407,4,7
7,807,8,7
8,1205,12,5
9,1307,13,7
10,1304,13,4


In [17]:
# Replace the values of 'COMID\SLC' with a sequence of numbers
slc_transposed['SLC'] = range(1, len(slc_transposed) + 1)

In [18]:
slc_transposed

,SLC,LULC,SOIL TYPE
1,1,10,7
2,2,10,4
3,3,12,7
4,4,12,4
5,5,10,5
6,6,4,7
7,7,8,7
8,8,12,5
9,9,13,7
10,10,13,4


In [19]:
#amount of rows for the rest of the data
array_rows=53

In [20]:
# Numbers to fill the columns with
fill_numbers = [0, 0, 0, 1, 0, 0, 2.296, 3, 0.091, 0.493, 2.296]

In [21]:
# Create a dictionary with column names and fill numbers
extra_data = {'Main crop cropid': [fill_numbers[0]] * array_rows,
        'Second crop cropid': [fill_numbers[1]] * array_rows,
        'Crop rotation group': [fill_numbers[2]] * array_rows,
        'Vegetation type': [fill_numbers[3]] * array_rows,
        'Special class code': [fill_numbers[4]] * array_rows,
        'Tile depth': [fill_numbers[5]] * array_rows,
        'Stream depth': [fill_numbers[6]] * array_rows,
        'Number of soil layers': [fill_numbers[7]] * array_rows,
        'Soil layer depth 1': [fill_numbers[8]] * array_rows,
        'Soil layer depth 2': [fill_numbers[9]] * array_rows,
        'Soil layer depth 3': [fill_numbers[10]] * array_rows}

In [22]:
# Create the DataFrame
full_data = pd.DataFrame(extra_data)

In [23]:
full_data

,Main crop cropid,Second crop cropid,Crop rotation group,Vegetation type,Special class code,Tile depth,Stream depth,Number of soil layers,Soil layer depth 1,Soil layer depth 2,Soil layer depth 3
0,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
1,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
2,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
3,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
4,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
5,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
6,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
7,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
8,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
9,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296


In [24]:
# Merge the two DataFrames
final_geoclass = pd.merge(slc_transposed, full_data, left_index=True, right_index=True)

In [25]:
final_geoclass

,SLC,LULC,SOIL TYPE,Main crop cropid,Second crop cropid,Crop rotation group,Vegetation type,Special class code,Tile depth,Stream depth,Number of soil layers,Soil layer depth 1,Soil layer depth 2,Soil layer depth 3
1,1,10,7,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
2,2,10,4,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
3,3,12,7,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
4,4,12,4,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
5,5,10,5,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
6,6,4,7,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
7,7,8,7,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
8,8,12,5,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
9,9,13,7,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296
10,10,13,4,0,0,0,1,0,0,2.296,3,0.091,0.493,2.296


In [26]:
# Output merged DataFrame to a .txt file
output_file = '/home/paulc600/local/HYPE Inputs/GeoClass.txt'

In [27]:
# Add commented lines
commented_lines = [
"""! MODIS landcover													
! Add legend (raster value) and discription													
!	original legend (raster_value)	description											
!	1	Evergreen Needleleaf Forests 											
!	2	Evergreen Broadleaf Forests 											
!	3	Deciduous Needleleaf Forests 											
!	4	Deciduous Broadleaf Forests 											
!	5	Mixed Forests 											
!	6	Closed Shrublands 											
!	7	Open Shrublands 											
!	8	Woody Savannas 											
!	9	Savannas 											
!	10	Grasslands 											
!	11	Permanent Wetlands 											
!	12	Croplands 											
!	13	Urban and Built-up Lands 											
!	14	Cropland/ Natural Vegetation Mosmcs 											
!	15	Permanent Snow and Ice 											
!	16	Barren 											
!	17	Water Bodies 											
!													
!													
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------													
!	SoilGrid V1												
!		original legend (raster_value)	description										
!	 C 	1	 clay										
!	 SIC 	2	 silty clay										
!	 SC 	3	 sandy clay										
!	 CL 	4	 clay loam										
!	 SICL 	5	 silty clay loam										
!	 SCL 	6	 sandy clay loam										
!	 L 	7	 loam										
!	 SIL 	8	 silty loam										
!	 SL 	9	 sandy loam										
!	 SI 	10	 silt										
!	 LS 	11	 loamy sand										
!	 S 	12	  sand										
!													
!													
! ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	
!          SLC	LULC	SOIL TYPE	Main crop cropid	Second crop cropid	Crop rotation group	Vegetation type	Special class code	Tile depth	Stream depth	Number of soil layers	Soil layer depth 1	Soil layer depth 2	Soil layer depth 3"""
]

In [28]:
# Open the file in write mode
with open(output_file, 'w') as file:
    # Write the commented lines
    for line in commented_lines:
        file.write(line + '\n')

In [29]:
with open(output_file, 'a') as file:
        final_geoclass.to_csv(file, sep='\t', index=False, header=False, line_terminator='\n')